In [1]:
!pip install diffusers
!pip install git+https://github.com/huggingface/accelerate.git
!pip install colabcode
!pip install fastapi
!pip install PythonVideoConverter

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-me9yx55x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-me9yx55x
  Resolved https://github.com/huggingface/accelerate.git to commit 9964f90fd7d50577998a22f3dba8590e644d255b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.25.0.dev0-py3-none-any.whl size=267070 sha256=877b6cf02eae01b0e54b13e46bd8a2fe153a4a4180a1fb2cbc94c21d6d69832d
  Stored in directory: /tmp/pip-ephem-wheel-cache-9er9559j/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Using cached colabcode-0.3.0-py3-none-any.whl (5.0 kB)
  Using cached pyngrok-7.0.3-py3-none-any.whl (21 kB)
  Using cached nest_asyncio-1.4.3-py3-none-any.whl (5.3 kB)
  Using cached uvicorn-0.13.1-py3-none-any.whl (4

In [8]:
from colabcode import ColabCode

from fastapi import FastAPI, Request
from fastapi.responses import FileResponse

from converter import Converter

conv = Converter()

import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

In [3]:
cc = ColabCode(port=12000, code=False)


In [4]:
pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1)
pipe.enable_vae_slicing()


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors not found


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/681M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
app = FastAPI()

@app.get("/")
async def inference(request:Request):
  prompt = request.query_params.get('prompt')
  num_frames = request.query_params.get('num_frames')
  video_frames = pipe(prompt, num_frames=int(num_frames)).frames
  video_path = export_to_video(video_frames)

  convert = conv.convert(video_path, video_path+'out', {
            'format': 'mp4',
            'video': {
                'codec': 'h264'
            }
        })# to make the video compatible with streamlit
  for timecode in convert:
      pass

  return FileResponse(video_path+'out', media_type="video/mp4")

In [10]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://6a0e-34-125-56-77.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [680]
INFO:uvicorn.error:Started server process [680]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     41.141.220.119:0 - "GET /?prompt=spider&num_frames=10 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [680]
INFO:uvicorn.error:Finished server process [680]
